# Integer & Linear Programming

## An example



Setting up data: cost matrix, demand, supply

In [ ]:
F = range(2) # 2 factories
R = range(12) # 12 retailers
C = [[1,2,2,1,3,4,5,7,5,2,3,2],
    [4,5,5,4,1,3,1,2,1,2,4,6]]
demand = [9,4,2,6,4,5,7,8,3,6,9,5]
print('Total demand =',sum(demand))
supply = [ 34, 45]

Now we can define a linear program 
$$\min \sum_{i\in F}\sum_{j\in R} c_{ij} x_{ij}$$
Subject To
$$\sum_{r\in R} x_{fr} \le s_r\quad\forall f\in F$$
$$\sum_{f\in F} x_{fr} = d_f\quad\forall r\in R$$
$$x \ge 0$$

In [ ]:
from mymip.mycplex import Model

lp = Model()
# define double indexed variables and give them a meaningful names
x = [ [lp.variable('x%dto%d'%(i,j)) for j in R] 
     for i in F ]

lp.min( sum( C[i][j] * x[i][j] for i in F for j in R))
# constraints can be given names too:
lp.SubjectTo({"Supply%d"%f:   sum(x[f][r] for r in R) <= supply[f]   for f in F})
lp.SubjectTo(("Demand%02d"%r, sum(x[f][r] for f in F) == demand[r] ) for r in R)
for f in F: 
    for r in R: x[f][r] >= 0   # all variables non-negative
lp.param["SCRIND"]=1    # set parameter to show CPLEX output 
lp.optimise()

print("The minimum cost is",lp.objective())
for r in R:
    for f in F:
        if x[f][r].x > 0: # amount is not zero
            print("%.1f from F%d to R%02d"%(x[f][r].x,f,r))

To see how the solve sees this problem, try writing it out to file and printing the contents of the file:

In [ ]:
lp.write("myfirst.lp")
print(open("myfirst.lp","r").read())

## Advanced Usage
All of the raw CPLEX callable library functions as per the can be accessed if required using C interface (see [IBM's CPLEX documentation](http://www.ibm.com/support/knowledgecenter/SSSA5P_12.6.2/ilog.odms.cplex.help/refcallablelibrary/groups/homepagecallable.html)) can be accessed by using the `cplex` object. 

For example to identify a minimal conflict set leading to infeasiblity of the problem we can use the [CPXrefineconflict](http://www.ibm.com/support/knowledgecenter/SSSA5P_12.6.2/ilog.odms.cplex.help/refcallablelibrary/cpxapi/refineconflict.html) and write out the conflict set using [CPXclpwrite](http://www.ibm.com/support/knowledgecenter/en/SSSA5P_12.6.2/ilog.odms.cplex.help/refcallablelibrary/cpxapi/clpwrite.html)

In [ ]:
from mymip.mycplex import cplex
lp.SubjectTo(x[0][0]+x[1][1] >= 50) # make it infeasible
lp.optimise() # error!
cplex.CPXrefineconflict(lp.Env,lp.LP,0,0) # 2 null pointers
cplex.CPXclpwrite(lp.Env,lp.LP,b"conflict.lp") # note binary (ascii) string
print("#"*10,"Conflict LP","#"*10)
print(open("conflict.lp","r").read())